In [1]:
from upapy.io import load_data
from upapy.reconstruction.ubp2d import FastUBP
from focus_func import *

In [2]:
# 批量重构函数
def batch_reconstruct(
    data: np.ndarray, sos: float = 1.5, image_size: int = 30, half_time: int = 0
) -> np.ndarray:
    if data.ndim == 3:
        fubp = FastUBP()
        fubp.set_pa_frame_element_number(data.shape[1])
        fubp.set_reconstruction_water_sos(sos)
        fubp.set_reconstruction_tissue_sos(sos)
        fubp.set_image_length(image_size)
        fubp.set_image_width(image_size)
        fubp.set_control_half_time_delta(half_time)
        recon_list = []
        for i in range(data.shape[0]):
            recon_slice = fubp.reconstruction(data[i])
            recon_list.append(recon_slice)
        return np.array(recon_list)
    else:
        raise ValueError("Incorrect array dimension.")

In [3]:
data = load_data("../data/mouse_data/D11980_1064 nm_30 %_20db_mouse_000.pah5")
data = data[50:150]
data.shape

(100, 512, 2000)

In [4]:
# 定义目标函数
def objective_function(sos):
    recon = batch_reconstruct(data, sos)
    mip = np.max(-recon, axis=0)
    gradient = calculate_tenenbaum_gradient(mip[np.newaxis,])

    return -gradient

In [5]:
import math

def golden_section_search(f, a, b, tol=1e-5, max_iter=100):
    gr = (math.sqrt(5) + 1) / 2  # 黄金比例 ≈ 1.618

    c = b - (b - a) / gr
    d = a + (b - a) / gr

    fc = f(c)
    fd = f(d)
    num_calls = 2

    for _ in range(max_iter):
        if abs(b - a) < tol:
            break

        if fc < fd:
            b, fd = d, fc
            d = c
            c = b - (b - a) / gr
            fc = f(c)
        else:
            a, fc = c, fd
            c = d
            d = a + (b - a) / gr
            fd = f(d)
        num_calls += 1

    x_min = (b + a) / 2
    f_min = f(x_min)
    num_calls += 1

    return x_min, f_min, num_calls

x_min, f_min, num_calls = golden_section_search(objective_function, 1.45, 1.55, tol=0.0005)
print(x_min, f_min, num_calls)

1.5093275563764617 [-50539840.] 15
